### Notebook Goals
* O objetivo deste notebook é obter mensagens da rede social Twitter, para analisar o sentimento geral sobre um produto na lingua portuguesa (pt).
* Neste caso será considerado o produto 'livros'.
* Para esse cálculo será utilizado inteligência artifical com NLP.

#### NLP
* NLP (Natural Linguage Processing) explora habilidades computacionais para entender e classificar textos.

##### Data Prep

In [1]:
import tweepy
import pandas as pd
import nltk #Natural Linguage Toolkit

In [2]:
#Import text file keys 
txtKeys = open('ktw.txt', 'r').read().splitlines()
api_key = txtKeys[0]
api_key_secret = txtKeys[1]
api_token = txtKeys[2]
api_token_secret = txtKeys[3]

In [3]:
#Twitter API authentication 
auth = tweepy.OAuthHandler(consumer_key=api_key, consumer_secret=api_key_secret)
auth.set_access_token(api_token, api_token_secret)
api = tweepy.API(auth)

In [4]:
#API params
search_query = 'livro' + ' -filter:retweets'
tweet_amount = 200
tweets = tweepy.Cursor(api.search, q=search_query, lang='pt').items(tweet_amount)

In [5]:
#Tweets list
cols = set()
allowed_types = [str, int]
tw_data = []

for tw in tweets:
    tw_dict = dict(vars(tw))
    tw_keys = tw_dict.keys()
    filter_data = {}
    for k in tw_keys:
        try:
            k_type = type(tw_dict[k])
        except:
            k_type = None
        if k_type != None:
            if k_type in allowed_types:
                cols.add(k)
                filter_data[k] = tw_dict[k]
    tw_data.append(filter_data)

header_cols = list(cols)
#tw_data

In [6]:
#Tweets Dataframe
df = pd.DataFrame(tw_data, columns=header_cols)
#df.head()
df.columns

Index(['retweet_count', 'quoted_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'lang', 'in_reply_to_screen_name',
       'favorite_count', 'source_url', 'quoted_status_id', 'id_str',
       'in_reply_to_status_id', 'id', 'source', 'text',
       'in_reply_to_status_id_str'],
      dtype='object')

In [7]:
#Cleaning Data
df.drop(['in_reply_to_status_id_str', 'in_reply_to_screen_name',
       'in_reply_to_user_id', 'id_str', 'in_reply_to_status_id',
       'favorite_count', 'in_reply_to_user_id_str', 'lang', 'source',
       'source_url']
 , inplace=True
 , axis=1)

In [8]:
df = df[['id', 'text', 'retweet_count']]

In [9]:
df.head()

,id,text,retweet_count
0,1390266893953609737,@meu_livro_off Nando,0
1,1390266889776123906,- Qual seu livro favorito?\n- Trono de Vidro \...,0
2,1390266888790368261,sempre caio nessa trend do booktok das pessoas...,0
3,1390266885338435589,@MauricioNardiJ6 @NelsonY2 @brumelianebrum Lei...,0
4,1390266877486718984,@CaioCorraini foda q o resumo do livro é: pagu...,0


##### NLP

In [ ]:
#Regular Expressions

In [ ]:
#Stopwords

In [ ]:
#Stemming

In [ ]:
#Classification